# 사이드스캔 소나 기물 탐지 - 탐색적 분석

이 노트북에서는 샘플 데이터를 사용하여 다음 작업을 수행합니다:

1. **데이터 로드 및 파싱**: XTF 파일 읽기 및 기본 정보 확인
2. **좌표 매핑**: 기물 위치와 소나 데이터 매핑
3. **전처리 파이프라인**: 워터컬럼 제거, 정규화, 노이즈 제거
4. **시각화**: 처리 결과 시각화 및 검증
5. **기물 탐지**: 기물 위치 표시 및 바운딩 박스 생성

## 환경 설정

In [ ]:
# 필요한 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# 프로젝트 경로 설정
project_root = Path('..')
sys.path.append(str(project_root))

# 프로젝트 모듈 import
from src.data_processing.xtf_reader import XTFReader, BatchXTFProcessor
from src.data_processing.coordinate_mapper import (
    CoordinateTransformer, 
    TargetLocationLoader, 
    CoordinateMapper
)
from src.data_processing.preprocessor import (
    Preprocessor, 
    PreprocessingConfig, 
    TerrainType
)
from config.settings import *
from config.paths import path_manager

# 시각화 설정
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("환경 설정 완료!")

## 1. 샘플 데이터 로드 및 분석

In [ ]:
# 샘플 데이터 파일 확인
sample_files = path_manager.list_sample_files()
print("샘플 파일 목록:")
for filename, filepath in sample_files.items():
    print(f"  - {filename}: {filepath.stat().st_size / (1024*1024):.1f} MB")

In [ ]:
# XTF 파일 로드
xtf_filename = XTF_CONFIG['sample_file']
xtf_filepath = path_manager.get_sample_file(xtf_filename)

print(f"XTF 파일 로드 중: {xtf_filename}")

# XTF Reader 초기화 (메모리 효율성을 위해 ping 수 제한)
xtf_reader = XTFReader(
    filepath=xtf_filepath, 
    max_pings=XTF_CONFIG['max_pings_per_load']
)

# 파일 로드 및 파싱
if xtf_reader.load_file():
    ping_data = xtf_reader.parse_pings()
    print("XTF 파일 로드 성공!")
    
    # 기본 정보 출력
    summary = xtf_reader.get_summary()
    print(f"\n=== XTF 파일 요약 ===")
    print(f"총 ping 수: {summary['total_pings']}")
    print(f"소나 채널 수: {summary['num_sonar_channels']}")
    print(f"주파수 정보: {summary['frequency_info']}")
    print(f"좌표 범위: {summary['coordinate_bounds']}")
else:
    print("XTF 파일 로드 실패!")

In [ ]:
# ping 데이터 분석
if ping_data:
    # 위치 정보 데이터프레임 생성
    geo_df = xtf_reader.get_georeferenced_data()
    
    print(f"\n=== Ping 데이터 분석 ===")
    print(f"데이터프레임 크기: {geo_df.shape}")
    print(f"\n컬럼 정보:")
    print(geo_df.info())
    
    print(f"\n위치 통계:")
    print(geo_df[['latitude', 'longitude']].describe())
    
    # 채널별 데이터 분석
    channel_counts = geo_df['channel'].value_counts()
    print(f"\n채널별 ping 수:")
    print(channel_counts)

## 2. Intensity 데이터 분석

In [ ]:
# 채널별 intensity 매트릭스 추출
port_intensity, port_geo = xtf_reader.get_channel_data(XTF_CONFIG['channels']['port'])
starboard_intensity, starboard_geo = xtf_reader.get_channel_data(XTF_CONFIG['channels']['starboard'])

print(f"Port 채널 intensity shape: {port_intensity.shape}")
print(f"Starboard 채널 intensity shape: {starboard_intensity.shape}")

# Intensity 통계 분석
print(f"\n=== Intensity 통계 (Port) ===")
print(f"최솟값: {np.min(port_intensity):.2f}")
print(f"최댓값: {np.max(port_intensity):.2f}")
print(f"평균: {np.mean(port_intensity):.2f}")
print(f"표준편차: {np.std(port_intensity):.2f}")

print(f"\n=== Intensity 통계 (Starboard) ===")
print(f"최솟값: {np.min(starboard_intensity):.2f}")
print(f"최댓값: {np.max(starboard_intensity):.2f}")
print(f"평균: {np.mean(starboard_intensity):.2f}")
print(f"표준편차: {np.std(starboard_intensity):.2f}")

In [ ]:
# 원본 intensity 데이터 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Port 채널 이미지
im1 = axes[0, 0].imshow(port_intensity, aspect='auto', cmap='gray')
axes[0, 0].set_title('Port 채널 (원본)')
axes[0, 0].set_xlabel('샘플 번호')
axes[0, 0].set_ylabel('Ping 번호')
plt.colorbar(im1, ax=axes[0, 0])

# Starboard 채널 이미지
im2 = axes[0, 1].imshow(starboard_intensity, aspect='auto', cmap='gray')
axes[0, 1].set_title('Starboard 채널 (원본)')
axes[0, 1].set_xlabel('샘플 번호')
axes[0, 1].set_ylabel('Ping 번호')
plt.colorbar(im2, ax=axes[0, 1])

# Port 채널 히스토그램
axes[1, 0].hist(port_intensity.flatten(), bins=100, alpha=0.7, color='blue')
axes[1, 0].set_title('Port 채널 Intensity 분포')
axes[1, 0].set_xlabel('Intensity 값')
axes[1, 0].set_ylabel('빈도')
axes[1, 0].grid(True)

# Starboard 채널 히스토그램
axes[1, 1].hist(starboard_intensity.flatten(), bins=100, alpha=0.7, color='red')
axes[1, 1].set_title('Starboard 채널 Intensity 분포')
axes[1, 1].set_xlabel('Intensity 값')
axes[1, 1].set_ylabel('빈도')
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig(path_manager.figures / '01_raw_intensity_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. 좌표 매핑 및 기물 위치 로드

In [ ]:
# 좌표 변환기 초기화
coord_transformer = CoordinateTransformer(utm_zone=COORDINATE_CONFIG['utm_zone'])

# 기물 위치 로더 초기화
target_loader = TargetLocationLoader(coord_transformer)

# 엑셀 파일에서 기물 위치 정보 로드
location_file = path_manager.get_sample_file(COORDINATE_CONFIG['location_file'])

if location_file.exists():
    print(f"기물 위치 파일 로드 중: {location_file.name}")
    
    # 엑셀 파일 구조 먼저 확인
    df_preview = pd.read_excel(location_file)
    print(f"\n엑셀 파일 구조 (처음 5행):")
    print(df_preview.head())
    print(f"\n컬럼 목록: {list(df_preview.columns)}")
    
    # 실제 컬럼명에 맞춰 로드 (컬럼명을 확인 후 수정 필요)
    # 예시: 실제 컬럼명에 따라 수정
    success = target_loader.load_from_excel(
        location_file, 
        lat_col='latitude',  # 실제 컬럼명으로 변경 필요
        lon_col='longitude'  # 실제 컬럼명으로 변경 필요
    )
    
    if success:
        targets_df = target_loader.get_targets_dataframe()
        print(f"\n기물 위치 로드 성공: {len(targets_df)} 개")
        print(targets_df.head())
    else:
        print("기물 위치 로드 실패 - 컬럼명을 확인하세요")
        targets_df = pd.DataFrame()
else:
    print(f"기물 위치 파일을 찾을 수 없습니다: {location_file}")
    targets_df = pd.DataFrame()

In [ ]:
# 좌표 매핑기 초기화 (Port 채널 사용)
coord_mapper = CoordinateMapper(coord_transformer)

# 소나 데이터 설정
coord_mapper.set_sonar_data(
    ping_coordinates=port_geo[['latitude', 'longitude', 'ping_number']],
    intensity_shape=port_intensity.shape
)

print("좌표 매핑기 설정 완료")

# 샘플 좌표 변환 테스트
if not targets_df.empty:
    # 첫 번째 기물의 픽셀 좌표 변환
    first_target = target_loader.target_locations[0]
    pixel_coords = coord_mapper.geo_to_pixel(first_target.longitude, first_target.latitude)
    
    print(f"\n좌표 변환 테스트:")
    print(f"기물 위경도: ({first_target.longitude:.6f}, {first_target.latitude:.6f})")
    print(f"픽셀 좌표: {pixel_coords}")
    
    # 역변환 테스트
    if pixel_coords[0] >= 0 and pixel_coords[1] >= 0:
        reverse_coords = coord_mapper.pixel_to_geo(pixel_coords[0], pixel_coords[1])
        print(f"역변환 결과: ({reverse_coords[0]:.6f}, {reverse_coords[1]:.6f})")

## 4. 전처리 파이프라인 적용

In [ ]:
# 전처리 설정
preprocess_config = PreprocessingConfig(
    remove_water_column=True,
    water_column_width=50,
    normalize_intensity=True,
    normalization_method='minmax',
    apply_denoising=True,
    denoising_method='gaussian',
    enhance_contrast=True,
    contrast_method='clahe',
    terrain_adaptive=True
)

# 전처리기 초기화
preprocessor = Preprocessor(preprocess_config)

print("전처리기 초기화 완료")

In [ ]:
# Port 채널 전처리
print("Port 채널 전처리 시작...")
port_result = preprocessor.process(port_intensity)

print(f"전처리 완료!")
print(f"처리 단계: {port_result.processing_steps}")
print(f"품질 메트릭: {port_result.quality_metrics}")

# Starboard 채널 전처리
print("\nStarboard 채널 전처리 시작...")
starboard_result = preprocessor.process(starboard_intensity)

print(f"전처리 완료!")
print(f"처리 단계: {starboard_result.processing_steps}")
print(f"품질 메트릭: {starboard_result.quality_metrics}")

In [ ]:
# 전처리 결과 시각화
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Port 채널 비교
im1 = axes[0, 0].imshow(port_intensity, aspect='auto', cmap='gray')
axes[0, 0].set_title('Port 채널 (원본)')
plt.colorbar(im1, ax=axes[0, 0])

im2 = axes[0, 1].imshow(port_result.processed_data, aspect='auto', cmap='gray')
axes[0, 1].set_title('Port 채널 (전처리 후)')
plt.colorbar(im2, ax=axes[0, 1])

# Port 채널 지형 맵 (있는 경우)
if port_result.terrain_map is not None:
    # 지형 맵을 숫자로 변환하여 시각화
    terrain_numeric = np.zeros_like(port_result.terrain_map, dtype=int)
    for i, terrain_type in enumerate([TerrainType.SAND.value, TerrainType.MUD.value, 
                                    TerrainType.ROCK.value, TerrainType.MIXED.value]):
        terrain_numeric[port_result.terrain_map == terrain_type] = i
    
    im3 = axes[0, 2].imshow(terrain_numeric, aspect='auto', cmap='tab10')
    axes[0, 2].set_title('Port 채널 지형 분류')
    plt.colorbar(im3, ax=axes[0, 2])
else:
    axes[0, 2].text(0.5, 0.5, '지형 맵 없음', transform=axes[0, 2].transAxes, 
                   ha='center', va='center')
    axes[0, 2].set_title('지형 분류 (없음)')

# Port 채널 히스토그램 비교
axes[0, 3].hist(port_intensity.flatten(), bins=50, alpha=0.5, label='원본', density=True)
axes[0, 3].hist(port_result.processed_data.flatten(), bins=50, alpha=0.5, label='전처리 후', density=True)
axes[0, 3].set_title('Port 채널 분포 비교')
axes[0, 3].legend()
axes[0, 3].grid(True)

# Starboard 채널 비교
im4 = axes[1, 0].imshow(starboard_intensity, aspect='auto', cmap='gray')
axes[1, 0].set_title('Starboard 채널 (원본)')
plt.colorbar(im4, ax=axes[1, 0])

im5 = axes[1, 1].imshow(starboard_result.processed_data, aspect='auto', cmap='gray')
axes[1, 1].set_title('Starboard 채널 (전처리 후)')
plt.colorbar(im5, ax=axes[1, 1])

# Starboard 채널 지형 맵
if starboard_result.terrain_map is not None:
    terrain_numeric = np.zeros_like(starboard_result.terrain_map, dtype=int)
    for i, terrain_type in enumerate([TerrainType.SAND.value, TerrainType.MUD.value, 
                                    TerrainType.ROCK.value, TerrainType.MIXED.value]):
        terrain_numeric[starboard_result.terrain_map == terrain_type] = i
    
    im6 = axes[1, 2].imshow(terrain_numeric, aspect='auto', cmap='tab10')
    axes[1, 2].set_title('Starboard 채널 지형 분류')
    plt.colorbar(im6, ax=axes[1, 2])
else:
    axes[1, 2].text(0.5, 0.5, '지형 맵 없음', transform=axes[1, 2].transAxes, 
                   ha='center', va='center')
    axes[1, 2].set_title('지형 분류 (없음)')

# Starboard 채널 히스토그램 비교
axes[1, 3].hist(starboard_intensity.flatten(), bins=50, alpha=0.5, label='원본', density=True)
axes[1, 3].hist(starboard_result.processed_data.flatten(), bins=50, alpha=0.5, label='전처리 후', density=True)
axes[1, 3].set_title('Starboard 채널 분포 비교')
axes[1, 3].legend()
axes[1, 3].grid(True)

plt.tight_layout()
plt.savefig(path_manager.figures / '02_preprocessing_results.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. 기물 탐지 및 시각화

In [ ]:
# 기물 마스크 및 바운딩 박스 생성
if not targets_df.empty and target_loader.target_locations:
    print(f"기물 마스크 생성 중... ({len(target_loader.target_locations)} 기물)")
    
    # 현재 데이터 영역 내의 기물만 필터링
    coord_bounds = xtf_reader.get_summary()['coordinate_bounds']
    targets_in_area = target_loader.get_targets_in_bounds(
        min_lat=coord_bounds['lat'][0],
        max_lat=coord_bounds['lat'][1],
        min_lon=coord_bounds['lon'][0],
        max_lon=coord_bounds['lon'][1]
    )
    
    print(f"데이터 영역 내 기물 수: {len(targets_in_area)}")
    
    if targets_in_area:
        # Port 채널용 마스크 생성
        port_mask = coord_mapper.create_target_mask(targets_in_area, mask_radius=5)
        
        # 바운딩 박스 생성
        bounding_boxes = coord_mapper.get_target_bounding_boxes(targets_in_area, box_size=20)
        
        print(f"마스크 생성 완료 - {np.sum(port_mask)} 픽셀")
        print(f"바운딩 박스 {len(bounding_boxes)}개 생성")
        
        # 바운딩 박스 정보 출력
        if bounding_boxes:
            bbox_df = pd.DataFrame(bounding_boxes)
            print("\n바운딩 박스 정보:")
            print(bbox_df[['target_id', 'center_x', 'center_y', 'width', 'height']].head())
    else:
        print("데이터 영역 내에 기물이 없습니다.")
        port_mask = np.zeros(port_intensity.shape, dtype=np.uint8)
        bounding_boxes = []
else:
    print("기물 위치 정보가 없습니다.")
    port_mask = np.zeros(port_intensity.shape, dtype=np.uint8)
    bounding_boxes = []

In [ ]:
# 기물 탐지 결과 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 원본 데이터와 기물 위치
im1 = axes[0, 0].imshow(port_intensity, aspect='auto', cmap='gray')
axes[0, 0].set_title('원본 데이터')
plt.colorbar(im1, ax=axes[0, 0])

# 전처리된 데이터와 기물 위치
im2 = axes[0, 1].imshow(port_result.processed_data, aspect='auto', cmap='gray')
axes[0, 1].set_title('전처리된 데이터')
plt.colorbar(im2, ax=axes[0, 1])

# 기물 마스크
im3 = axes[1, 0].imshow(port_mask, aspect='auto', cmap='Reds')
axes[1, 0].set_title(f'기물 마스크 ({np.sum(port_mask)} 픽셀)')
plt.colorbar(im3, ax=axes[1, 0])

# 전처리된 데이터 + 바운딩 박스
im4 = axes[1, 1].imshow(port_result.processed_data, aspect='auto', cmap='gray')
axes[1, 1].set_title(f'전처리 데이터 + 바운딩 박스 ({len(bounding_boxes)}개)')

# 바운딩 박스 그리기
for bbox in bounding_boxes[:10]:  # 최대 10개만 표시
    from matplotlib.patches import Rectangle
    rect = Rectangle(
        (bbox['x1'], bbox['y1']), 
        bbox['width'], 
        bbox['height'],
        linewidth=2, 
        edgecolor='red', 
        facecolor='none'
    )
    axes[1, 1].add_patch(rect)
    
    # 기물 ID 표시
    axes[1, 1].text(
        bbox['center_x'], 
        bbox['center_y'], 
        bbox['target_id'], 
        color='yellow',
        fontsize=8,
        ha='center',
        va='center'
    )

plt.colorbar(im4, ax=axes[1, 1])

for ax in axes.flat:
    ax.set_xlabel('샘플 번호')
    ax.set_ylabel('Ping 번호')

plt.tight_layout()
plt.savefig(path_manager.figures / '03_target_detection_results.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. 결과 저장 및 내보내기

In [ ]:
# 전처리된 데이터 저장
print("결과 저장 중...")

# 1. 전처리된 intensity 데이터 저장
np.save(path_manager.processed_data / 'port_processed_intensity.npy', port_result.processed_data)
np.save(path_manager.processed_data / 'starboard_processed_intensity.npy', starboard_result.processed_data)

# 2. 기물 마스크 저장
np.save(path_manager.processed_data / 'target_mask.npy', port_mask)

# 3. 바운딩 박스 정보 저장
if bounding_boxes:
    bbox_df = pd.DataFrame(bounding_boxes)
    bbox_df.to_csv(path_manager.processed_data / 'bounding_boxes.csv', index=False)

# 4. 위치 정보 저장
geo_df.to_csv(path_manager.processed_data / 'ping_coordinates.csv', index=False)

# 5. 처리 결과 메타데이터 저장
metadata = {
    'port_processing_steps': port_result.processing_steps,
    'port_quality_metrics': port_result.quality_metrics,
    'starboard_processing_steps': starboard_result.processing_steps,
    'starboard_quality_metrics': starboard_result.quality_metrics,
    'num_targets': len(bounding_boxes),
    'data_shape': port_intensity.shape,
    'processing_config': {
        'remove_water_column': preprocess_config.remove_water_column,
        'normalization_method': preprocess_config.normalization_method,
        'denoising_method': preprocess_config.denoising_method,
        'contrast_method': preprocess_config.contrast_method
    }
}

import json
with open(path_manager.processed_data / 'processing_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("저장 완료!")
print(f"저장 위치: {path_manager.processed_data}")

# 저장된 파일 목록 출력
saved_files = list(path_manager.processed_data.glob('*'))
print(f"\n저장된 파일들:")
for file in saved_files:
    print(f"  - {file.name} ({file.stat().st_size / 1024:.1f} KB)")

## 7. 분석 요약 및 다음 단계

In [ ]:
# 분석 요약 보고서 생성
print("=== 탐색적 분석 요약 ===")
print(f"\n1. 데이터 개요:")
print(f"   - XTF 파일: {xtf_filename}")
print(f"   - 총 ping 수: {len(ping_data)}")
print(f"   - Port 채널 크기: {port_intensity.shape}")
print(f"   - Starboard 채널 크기: {starboard_intensity.shape}")

print(f"\n2. 전처리 결과:")
print(f"   - Port SNR: {port_result.quality_metrics['snr']:.2f} dB")
print(f"   - Port 엣지 보존: {port_result.quality_metrics['edge_preservation']:.3f}")
print(f"   - Port 대비 개선: {port_result.quality_metrics['contrast_improvement']:.3f}")
print(f"   - Starboard SNR: {starboard_result.quality_metrics['snr']:.2f} dB")
print(f"   - Starboard 엣지 보존: {starboard_result.quality_metrics['edge_preservation']:.3f}")
print(f"   - Starboard 대비 개선: {starboard_result.quality_metrics['contrast_improvement']:.3f}")

print(f"\n3. 기물 탐지:")
print(f"   - 총 기물 수: {len(target_loader.target_locations) if target_loader.target_locations else 0}")
print(f"   - 데이터 영역 내 기물: {len(targets_in_area) if 'targets_in_area' in locals() else 0}")
print(f"   - 생성된 바운딩 박스: {len(bounding_boxes)}")
print(f"   - 마스크 픽셀 수: {np.sum(port_mask)}")

print(f"\n4. 다음 단계:")
print(f"   - 특징 추출 알고리즘 적용 (HOG, LBP, Gabor, SfS)")
print(f"   - 데이터 증강 기법 적용 (불균형 데이터 해결)")
print(f"   - CNN 기반 탐지 모델 학습")
print(f"   - 모의 데이터와 실 데이터 비교 분석")
print(f"   - 성능 평가 및 모델 최적화")

# 권장사항
print(f"\n5. 권장사항:")
if port_result.quality_metrics['snr'] < 10:
    print(f"   - SNR이 낮음 ({port_result.quality_metrics['snr']:.2f}dB) - 노이즈 제거 강화 필요")

if len(bounding_boxes) == 0:
    print(f"   - 기물이 탐지되지 않음 - 좌표 매핑 또는 데이터 영역 확인 필요")

if np.sum(port_mask) < 100:
    print(f"   - 기물 마스크가 매우 작음 - 마스크 반경 조정 또는 좌표 정확도 확인 필요")

print(f"\n분석 완료! 결과는 다음 위치에 저장되었습니다:")
print(f"  - 데이터: {path_manager.processed_data}")
print(f"  - 그림: {path_manager.figures}")